# Marine Biodiversity and Environmental Data Exploration with `mlcroissant`
This notebook provides a template for loading and exploring a dataset using the `mlcroissant` library.

### Dataset Source
The dataset source is provided via a Croissant schema URL.

In [ ]:
# Ensure `mlcroissant` library is installed
!pip install mlcroissant

## 1. Data Loading
Load metadata and records from the dataset using `mlcroissant`.

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Define the dataset URL
url = 'https://cdn.dev.senscience.cloud/portals/10.82843/pm80-mh77/fair2.json'

# Load the dataset
dataset = mlc.Dataset(url)
metadata = dataset.metadata
print(f"{metadata['name']}: {metadata['description']}")

## 2. Data Overview
Review available record sets, fields, and their IDs.

In [ ]:
# List all record sets in the dataset
record_sets = dataset.record_sets
for record_set in record_sets:
    print(f"Record Set ID: {record_set['@id']}")
    for field in record_set['field']:
        print(f"  Field ID: {field['@id']} - Name: {field['name']}")

## 3. Data Extraction
Load data from a specific record set into a DataFrame for analysis. Use the record set and field `@id`s from the overview.

In [ ]:
# Extract data from each record set
record_sets_to_extract = [
    'https://senscience.ai/frontiers/borja/README.csv',
    'https://senscience.ai/frontiers/borja/METADATA.csv'
]
dataframes = {}

for record_set_id in record_sets_to_extract:
    records = list(dataset.records(record_set=record_set_id))
    dataframes[record_set_id] = pd.DataFrame(records)

# Display columns of a specific DataFrame
print(dataframes['https://senscience.ai/frontiers/borja/README.csv'].columns.tolist())
dataframes['https://senscience.ai/frontiers/borja/README.csv'].head()

## 4. Exploratory Data Analysis (EDA)
Apply common data processing steps, such as filtering records based on specific criteria, normalizing numeric fields, and categorizing data. This section should include operations like removing outliers, transforming data distributions, or grouping data by key attributes to prepare it for further analysis.

In [ ]:
# Select a numeric field for analysis
numeric_field = 'https://senscience.ai/frontiers/borja/METADATA.csv/decimallatitude'

threshold = 43.4
filtered_df = dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][
    dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][numeric_field] > threshold
]
print(f"Filtered records with {numeric_field} > {threshold}:")
print(filtered_df.head())

# Normalize the numeric field
filtered_df[f"{numeric_field}_normalized"] = (
    (filtered_df[numeric_field] - filtered_df[numeric_field].mean()) /
    filtered_df[numeric_field].std()
)
print(f"Normalized {numeric_field} for filtered records:")
print(filtered_df[[numeric_field, f"{numeric_field}_normalized"]].head())

# Group data by a key attribute
group_field = 'https://senscience.ai/frontiers/borja/METADATA.csv/taxagroup'
if group_field in dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].columns:
    grouped_df = filtered_df.groupby(group_field).mean()
    print(f"Grouped data by {group_field}:")
    print(grouped_df.head())

## 5. Visualization
Visualize data distributions or relationships between fields in the dataset.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the distribution of a numeric field
plt.figure(figsize=(10, 6))
sns.histplot(filtered_df[numeric_field], bins=30, kde=True)
plt.title('Distribution of Decimal Latitude')
plt.xlabel('Decimal Latitude')
plt.ylabel('Frequency')
plt.show()

## 6. Conclusion
Summarize key findings and observations from the dataset exploration.

Through this exploration, we have loaded and examined a dataset related to marine biodiversity and environmental monitoring. We've filtered and normalized data, and visualized distributions of key parameters. This gives us a clearer understanding of the dataset's structure and the types of analyses that can be performed to extract insightful information.